In [1]:
import sys
sys.path.append('../')

In [2]:
# %%capture
%run common.ipynb
# importnb allows us to load the code from other notebooks

# from importnb import Notebook
# with Notebook(lazy=True):
#     # import
#     import OfficeAutomation.common as common
#     import OfficeAutomation.lamps as lamps
#     import OfficeAutomation.ac as ac
#     import OfficeAutomation.electrical as electrical

In [3]:
%run lamps.ipynb
%run ac.ipynb
%run electrical.ipynb

In [4]:
import crestdsl.model as crest
import crestdsl.model.api as api
import crestdsl.simulation as crestsim
import crestdsl.ui as ui

In [5]:
class OfficeLightController(crest.Entity):
    """
    has three settings: on/off/auto
    if the switch is auto it turns the lights on if 
    luminosity < threshold and presence.
    """
    presence = crest.Input(res_presence, "no presence")
    switch_in = crest.Input(res_onoffauto, "auto")
    luminosity_in = crest.Input(res_lux, 200)
    light_threshold = crest.Local(res_lux, 600)
    
    out = crest.Output(res_onoff_switch, value="off") # on/off
    
    state = current = crest.State()
    
    @crest.update(state=state, target=out)
    def update_output(self, dt):
        if self.switch_in.value == "auto":
            if self.presence.value == "detected" and self.luminosity_in.value < self.light_threshold.value:
                return "on"
            else:
                return "off"
        elif self.switch_in.value == "on":
            return "on"
        elif self.switch_in.value == "off":
            return "off"
        
class HallwayLightController(crest.Entity):
    presence = crest.Input(res_presence, "no presence")
    out = crest.Output(res_onoffauto, value="off") # on/off
    luminosity_in = crest.Input(res_lux, 200)
    light_threshold = crest.Local(res_lux, 600)
    
    state = current = crest.State()
    
    @crest.update(state=state, target=out)
    def update_output(self, dt):
        if self.presence.value == "detected" and self.luminosity_in.value < self.light_threshold.value:
            return "on"
        else:
            return "off"


In [6]:
class Room(crest.Entity):
    state = current = crest.State()
    
    
    # calculate the room's temperature
    windows = crest.Local(res_integer, 2)
    temperature_outside = crest.Input(res_celsius, 25)
    temperature = crest.Local(res_celsius, 25)
    volume = crest.Local(res_volume, 100)
        
    # subentities
    acunit = ACUnit()
    light_controls = OfficeLightController()
    electricity_splitter = Splitter()    
    
    emergency_electricity_splitter = Splitter()
    # this one will be set by update
    req_emergency_electricity_out = crest.Output(res_electricity, value=0)
    
    def __init__(self, volume=100, number_windows=0, number_lamps=0, number_emergency_lamps=0):
        self.volume.value = volume
        self.windows.value = number_windows
        
        # install lamps and emergency lamps
        installed_lamps = []
        for i in range(number_lamps):
            lamp = api.add(self, f"lamp_{i+1}", Lamp())
            installed_lamps.append(lamp)
            api.add(self, f"connect_lamp_{i+1}_switch", crest.Influence(source=self.light_controls.out, target=lamp.switch_in))
        self.electricity_splitter.connect_devices(installed_lamps + [self.acunit])

        emergency_lamps = []
        for i in range(number_emergency_lamps):
            em_lamp = api.add(self, f"emergency_lamp_{i+1}", EmergencyLamp())
            emergency_lamps.append(em_lamp)
        self.emergency_electricity_splitter.connect_devices(emergency_lamps)
        
        """Create nice pullup and relay influences without manually setting everything"""
        # create connections conveniently:
        api.pullup(ac_setting_in=self.acunit.setting_in, 
               presence=self.light_controls.presence, 
               lightswitch=self.light_controls.switch_in,
               light_outside=self.light_controls.luminosity_in)
        
        api.pullup(self.electricity_splitter.electricity_in,
                  self.electricity_splitter.req_electricity_out)
        
        api.pullup(emergency_electricity_in=self.emergency_electricity_splitter.electricity_in)
        
        api.dependencies( (self.req_electricity_out, self.ac_setting_in),
                           (self.req_electricity_out, self.lightswitch),
                           (self.req_electricity_out, self.presence))

    
    @crest.update(state=state, target=req_emergency_electricity_out)
    def connect_on_surcharge(self, dt):
        """Only draws electricity if the normal splitter has a surcharge."""
        if self.electricity_splitter.surcharge.value:
            return self.emergency_electricity_splitter.req_electricity_out.value
        else:
            return 0
    
    
    @crest.update(state=state, target=temperature)
    def update_temperature(self, dt):
        """
        This function is not correct. 
        It depends on the output of the AC and the difference between inside and outside temperate.
        However, as time progresses, the difference between inside and outside changes.
        Thus, the return values are different for different dt values.
        
        TODO: 
        Do a bivariate piecewise regression / interpolation on temperature difference & time values
        
        WORKAROUND:
        Assert that we simulate with an appropriate maximum time steps size (e.g. 60)
        """
        u_factor = 2.84 # u-factor tells us how how well the window insulates. 2.84 is pretty bad...

        air_mass_kg = self.volume.value * 1.293  # 1.293 kg / m^3
        
        # window watt
        temp_diff = self.temperature_outside.value - self.temperature.value
        # we assume all windows are 1.5m x 2m in size
        room_window_surface = self.windows.value * 1.5 * 2
        window_watt_added = room_window_surface * u_factor * temp_diff  
        total_added_watt = window_watt_added - self.acunit.output.value
        
        joules = total_added_watt * dt * 60  # dt is in minutes
        specific_heat = 718 # J / (kg * Kelvin)
        kelvin = joules / (air_mass_kg * specific_heat)
        return self.temperature.value + kelvin


# ui.plot(Room(number_lamps=2, number_emergency_lamps=2))


sim = crestsim.Simulator(Room(number_lamps=2, number_emergency_lamps=2))
sim.stabilise()
sim.advance(3)
sim.plot()


In [ ]:
# %%prun -s cumulative
class Floor(crest.Entity):
    state = current = crest.State()
    
    # inputs for temperature and light outside
    temperature_outside = crest.Input(res_celsius, 25)
    light_outside = crest.Input(res_lux, 500)

    
    electricity_in = crest.Input(resource=res_electricity, value=3000)  # the main input
    mains = Splitter()
    connect_mains = crest.Influence(source=electricity_in, target=mains.electricity_in)
    
    hallway = Room(volume=10*3*3, number_windows=1, number_lamps=6, number_emergency_lamps=0)
    office1 = Room(volume=10*5*3, number_windows=3, number_lamps=8, number_emergency_lamps=4)
    office2 = Room(volume=5*5*3, number_windows=1, number_lamps=4, number_emergency_lamps=4)
    office3 = Room(volume=5*5*3, number_windows=2, number_lamps=4, number_emergency_lamps=10)    
  
    # control lights in the hallway
    hallway_light_controls = HallwayLightController()
    connect_hallway_light_controls = crest.Influence(source=hallway_light_controls.out, target=hallway.lightswitch)
    
    @crest.update(state=state, target=hallway_light_controls.presence)
    def update_hallway_lights(self, dt):
        presence_on_floor = self.presence_hallway.value == "detected" or \
                            self.presence_office1.value == "detected" or \
                            self.presence_office2.value == "detected" or \
                            self.presence_office3.value == "detected"
        if presence_on_floor:
            return "detected"
        else:
            return "no presence"

    def __init__(self):
        # use splitter to connect devices
        self.mains.connect_devices([self.hallway, self.office1, self.office2, self.office3])
        
        # pullup presence settings for each room
        api.pullup(presence_hallway=self.hallway.presence, 
                   presence_office1=self.office1.presence, 
                   presence_office2=self.office2.presence, 
                   presence_office3=self.office3.presence)
        
        # pull up ac settings for each room
        api.pullup(ac_setting_hallway=self.hallway.ac_setting_in,
                   ac_setting_office1=self.office1.ac_setting_in, 
                   ac_setting_office2=self.office2.ac_setting_in, 
                   ac_setting_office3=self.office3.ac_setting_in)
        
        # pull up light switch
        api.pullup(lightswitch_office1=self.office1.lightswitch, 
                   lightswitch_office2=self.office2.lightswitch, 
                   lightswitch_office3=self.office3.lightswitch)
        
        
        # propagate outside temperature and light
        api.relay(
            (self.temperature_outside, self.hallway.temperature_outside),  (self.light_outside, self.hallway.light_outside),
            (self.temperature_outside, self.office1.temperature_outside),  (self.light_outside, self.office1.light_outside),
            (self.temperature_outside, self.office2.temperature_outside),  (self.light_outside, self.office2.light_outside),
            (self.temperature_outside, self.office3.temperature_outside),  (self.light_outside, self.office3.light_outside)
        )
    
ui.plot(Floor())


sim = crestsim.Simulator(Floor())
sim.stabilise()
sim.advance(3)
sim.plot()


